# Setting Up Encounters

Now that we have some player data, we can give users something to do. Same as with players, we'll start by creating some backend logic before creating a "UI" for the bot.

To start with, we'll make a Mob class for the player to fight. In addition to stat properties, we'll also start filling out methods- it can attack, take damage, and check if it's alive. In order to add a bit of randomness, we'll make attack damage a range using a random integer.

In [ ]:
from random import randrange


class Mob:
    def __init__(self):
        self.current_hp = 10
        self.attack = 1
        self.xp = 2

    def do_attack(self):
        return self.attack + randrange(2)

    def take_damage(self, damage):
        self.current_hp -= damage
        return self.is_alive()

    def is_alive(self):
        return self.current_hp > 0

Setting up the player is going to look very similar. We'll give the damage a bigger range because in this scenario we want the player to be able to win easily. If we were to refactor, we would probably create a base class with these properties and methods, and then have mobs and players inherit from that base class.

In [ ]:
from random import randrange


class Player:
    def __init__(self, id, xp=0, attack=0, hp=0):
        self.id = id
        self.xp = xp
        self.attack = attack
        self.max_hp = hp
        self.current_hp = hp

    def do_attack(self):
        return self.attack + randrange(4)

    def take_damage(self, damage):
        self.current_hp -= damage
        return self.is_alive()

    def is_alive(self):
        return self.current_hp > 0


Lastly, we''l create some classes around running the actual encounter. This will have three elements: and enum (something that allows us to create a group of values like a list of colors), an ActionResult class that holds our result type and text to send to the user, and the encounter class itself in order to hold the logic of actually running the encounter.

Because the `takeDamage()` function returns a boolean that denotes whether or not an entity survived, we can use that as a way to check the results of the action as well as changing the values, but for now we'll seperate the steps to make it clearer.

In [ ]:
from mob import Mob
from player import Player
from enum import Enum


class Result(Enum):
    CONTINUE = 1
    WIN = 2
    LOSE = 3


class ActionResult:

    def __init__(self, result: Results, text: str):
        self.result = result
        self.text = text


class Encounter:
    def __init__(self, player: Player):
        self.id = 0
        self.mob = Mob()
        self.player = player

    def playerAction(self):
        damage = self.player.do_attack()
        self.mob.take_damage(damage)
        mob_result = self.mob.take_damage(damage)
        if(mob_result):
            mob_damage = self.mob.do_attack()
            player_result = self.player.take_damage(mobDamage)
            if(player_result):
                return ActionResult(Result.CONTINUE, f"Player attacked for {damage}. Mob attacked with {mobDamage}.")
            else:
                return ActionResult(Result.LOSE, f"Player attacked for {damage}. Mob attacked with {mobDamage}. Player fainted.")
        else:
            return ActionResult(Result.WIN, f"Player attacked for {damage}. Mob fainted.")


Next we need to update our Game class. In addition to a group of players, we need to keep track of current encounters. We'll also add methods to create new encounters, allow players to take attack actions and to complete an existing encounter.

In [ ]:
from player import Player
from encounter import Encounter, Result


class Game:
    def __init__(self):
        self.players = {}
        self.encounters = {}

    def getPlayer(self, playerId):
        player = self.players.get(playerId)
        if player:
            return player
        else:
            self.players[playerId] = Player(playerId, 0, 2, 10)
            return self.players[playerId]

    def createEncounter(self, playerId):
        encounter = self.encounters.get(playerId)
        if encounter:
            return "Encounter already running!"
        else:
            player = self.getPlayer(playerId)
            self.encounters[playerId] = Encounter(player)
            return "Encounter created!"

    def playerAction(self, playerId):
        encounter = self.encounters.get(playerId)
        if encounter is None:
            self.createEncounter(playerId)

        result = encounter.playerAction()
        self.players[playerId] = encounter.player

        if result.result == Result.CONTINUE:
            self.encounters[playerId] = encounter
        else:
            self.completeEncounter(playerId)

        return result.text

    def completeEncounter(self, playerId):
        encounter = self.encounters.get(playerId)
        if encounter is not None:
            del self.encounters[playerId]

The last piece is to add all this to the bot. 

In [ ]:
@client.command(
    name="attack",
    description="Attack!!!!!!",
    scope=GUILD,
)
async def player_action(ctx: interactions.CommandContext):
    result = game.playerAction(str(ctx.member.id))
    await ctx.send(result)


@client.command(
    name="run",
    description="Run away",
    scope=GUILD,
)
async def complete_encounter(ctx: interactions.CommandContext):
    result = game.completeEncounter(str(ctx.member.id))
    await ctx.send(result)